
# Objectives

Fetch data from Kaggle and prepare it for further processes.

# Inputs

Kaggle JSON file - the authentication token.

Outputs

# Generate Dataset: inputs/datasets/malaria_dataset

Additional Comments | Insights | Conclusions
No additional comments.

In [14]:
%pip install -r/workspaces/Malaria-Detection/requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy
import os

## Change the working directory

In [16]:
current_dir = os.getcwd()
current_dir

'/workspaces/Malaria-Detection/jupyter_notebooks'

In [17]:
current_dir = os.getcwd()
current_dir

'/workspaces/Malaria-Detection/jupyter_notebooks'

# Install Keggle

In [18]:
%pip install kaggle==1.5.12


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

chmod: cannot access 'kaggle.json': No such file or directory


In [20]:
KaggleDatasetPath = "codeinstitute/malaria-cell-classification"
DestinationFolder = "inputs/malaria_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Traceback (most recent call last):
  File "/home/codespace/.python/current/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/home/codespace/.python/current/lib/python3.12/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/codespace/.python/current/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /workspaces/Malaria-Detection/jupyter_notebooks. Or use the environment method.
